In [4]:
import pandas as pd
import os

# CSV 파일 경로 (직접 경로로 바꿔주세요)
csv_path = "data/test_with_answer.csv"

# 출력 디렉토리 설정
output_dir = "docs/train_chunks"
os.makedirs(output_dir, exist_ok=True)

# 몇 행씩 나눌 것인지 설정
chunk_size = 500  # 필요에 따라 100, 1000 등으로 조정 가능

# CSV 불러오기
df = pd.read_csv(csv_path)

# 총 분할 수 계산
num_chunks = (len(df) + chunk_size - 1) // chunk_size

# 각 분할을 텍스트로 저장
for i in range(num_chunks):
    chunk = df.iloc[i * chunk_size : (i + 1) * chunk_size]
    lines = []
    for _, row in chunk.iterrows():
        err = row.get("err_sentence", "")
        cor = row.get("cor_sentence", "")
        if pd.notnull(err) and pd.notnull(cor):
            lines.append(f"입력: {err}\n출력: {cor}\n")

    file_path = os.path.join(output_dir, f"train_chunk_{i+1:02}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

print(f"{num_chunks}개의 파일이 '{output_dir}'에 저장되었습니다.")

22개의 파일이 'docs/train_chunks'에 저장되었습니다.


In [1]:
import asyncio
import pandas as pd
import json
from tqdm import tqdm
from itertools import cycle
from prompts.template_format import format_prompt
from optimizer.evaluator import compute_scores
from openai import AsyncOpenAI
from dotenv import load_dotenv
import nest_asyncio
import os

# 🧠 비동기 이벤트 루프 충돌 방지 (Jupyter 전용)
nest_asyncio.apply()
load_dotenv()

# 📦 API 키 최대 10개 로딩
API_KEYS = [os.getenv(f"UPSTAGE_API_KEY_{i}") for i in range(1, 11)]
API_KEYS = [k for k in API_KEYS if k]

clients = [
    AsyncOpenAI(api_key=key, base_url="https://api.upstage.ai/v1")
    for key in API_KEYS
]
client_cycle = cycle(clients)
semaphores = {client: asyncio.Semaphore(3) for client in clients}  # 키당 최대 동시 3건 제한

# 📄 템플릿 로드
graph_path = "data/prompt_graph.jsonl"
selected_template_id = "base_02"
with open(graph_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        if obj.get("id") == selected_template_id:
            template_str = obj["template"]
            break

# 🧪 데이터 로딩
df = pd.read_csv("data/test_with_answer.csv")
sample = [
    {"input": row["err_sentence"], "output": row["cor_sentence"]}
    for _, row in df.iterrows()
    if pd.notnull(row["err_sentence"]) and pd.notnull(row["cor_sentence"])
]

# 🚀 제한 포함된 LLM 호출
async def call_with_limit(client, messages, input_text):
    async with semaphores[client]:
        try:
            response = await client.chat.completions.create(
                model="solar-pro",
                messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"\n❌ API 호출 오류: {e} → 입력 문장: {input_text[:50]}...")
            return "[ERROR]"

# 📊 전체 평가 함수
async def evaluate_all_concurrent(batch_size=100):
    results = []
    success_count = 0
    error_count = 0

    async def worker(row):
        nonlocal success_count, error_count
        client = next(client_cycle)
        messages = format_prompt(template_str, row["input"])
        pred = await call_with_limit(client, messages, row["input"])
        if pred == "[ERROR]":
            error_count += 1
        else:
            success_count += 1
        return {"prediction": pred, "output": row["output"]}

    for i in tqdm(range(0, len(sample), batch_size), desc="🔁 전체 진행 상황"):
        batch = sample[i:i+batch_size]
        tasks = [worker(row) for row in batch]
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)

    # 🔍 점수 계산
    recalls = []
    for res in results:
        _, recall, _ = compute_scores(res["prediction"], res["output"])
        recalls.append(recall)

    avg_recall = sum(recalls) / len(recalls) if recalls else 0.0
    print(f"\n📊 [{selected_template_id}] 평균 Recall: {round(avg_recall, 4)}")
    print(f"✅ 성공 요청 수: {success_count}개")
    print(f"⚠️ 실패 요청 수: {error_count}개")

# ▶ 실행
await evaluate_all_concurrent(batch_size=100)

🔁 전체 진행 상황: 100%|██████████| 109/109 [25:00<00:00, 13.76s/it]



📊 [base_02] 평균 Recall: 0.9768
✅ 성공 요청 수: 10805개
⚠️ 실패 요청 수: 0개


In [4]:
import asyncio
import pandas as pd
import json
from tqdm import tqdm
from itertools import cycle
from prompts.template_format import format_prompt
from openai import AsyncOpenAI
from dotenv import load_dotenv
import nest_asyncio
import os

# 비동기 루프 적용 (Jupyter용)
nest_asyncio.apply()
load_dotenv()

# API 키 로딩
API_KEYS = [os.getenv(f"UPSTAGE_API_KEY_{i}") for i in range(1, 11)]
API_KEYS = [k for k in API_KEYS if k]

clients = [
    AsyncOpenAI(api_key=key, base_url="https://api.upstage.ai/v1")
    for key in API_KEYS
]
client_cycle = cycle(clients)
semaphores = {client: asyncio.Semaphore(3) for client in clients}  # 키당 최대 동시 3건 제한

# 템플릿 직접 입력
template_str = """다음 문장에서 **오탈자 / 문장부호 / 띄어쓰기** 오류를 모두 교정해줘.

📌 **교정 원칙**
1. **오탈자(철자 오류)**
- 자판 입력 실수나 자음/모음 혼동을 고쳐줘 (예: '됬다' → '됐다')
- 모양이나 발음이 비슷한 단어(예: 외/왜, 믈론/물론 등)를 고려하되,
  ① 철자 오류인지 확인하고, ② 문장 의미에 비추어 자연스럽고 적절한 단어일 경우에만 수정해
- 문맥상 부적절하거나 존재하지 않는 단어는 자연스러운 단어로 바꿔줘

2. **띄어쓰기**
- 의미 단위에 맞춰 문법적으로 띄어쓰기 교정
- 조사, 어미, 접사 사이를 임의로 띄우지 마.
- 단어를 붙여야 할 경우 정확한 어절 단위로 붙여줘.
- 문체나 표현 방식은 유지해줘.
- 아래 단어는 반드시 **띄어 써야 해**:
  · '조회 수', '전공 수업', '행동 강령', '수능 문학', '샤프 심', '백지 복습', '후 순위', '교직 이수', '배면 발광',
    '전면 발광', '지역 페이', '초등 교사', '가짜 뉴스', '자음군 단순화', '기숙 학원', '만국 공법', '공부 의욕', '노인 성비'
- 아래 예시는 혼동되는 띄어쓰기야. 올바르게 입력되면 수정하지 말고, 틀리면 수정해:
  · '방 파시면'
- 아래 단어는 반드시 **붙여 써야 해**:
  · '수시접수', '시험기간', '구조독해', '동해바다', '학교생활', '공부시간', '정정기간', '인구수', '의사분', '회전주기',
    '공부자극', '악성코드', '나비효과', '필기노트', '주격조사', '수학학원', '인지심리학', '대학과정'
- 관용적 표현은 붙여 써야 해: ‘해주셨다’, ‘도와주셔서’, ‘한밤중’
- ‘듯’ 표현은 붙여 써야 해: ‘맞는듯해’, ‘없는듯’, ‘괜찮을듯하다’
- ‘해가지고’는 붙여 써야 해: ‘바뀌어가지고’, ‘터득해가지고’
- 보조동사 ‘주다’는 붙여 써야 해: ‘도와주셨다’, ‘해주셔서’

3. **문장부호**
- 문장 끝에 종결 부호가 없을 경우 마침표(.)를 추가해줘
- 쉼표(,)는 원문에 있는 경우만 유지하고 새로 삽입하지 마
- 정식 문장부호가 아닌 (...), (...?), (..?), (...!) 등은 유지
- 그 외 문장부호 추가/변경 금지

📌 **교정 순서**
① 오탈자 → ② 띄어쓰기 → ③ 문장부호

📌 **금지 사항**
- 표현 방식, 말투, 격식 수준 변경 금지 (예: '선생' → '선생님' 금지)
- 같은 의미라도 조사/어미/단어 변경 금지 (예: '밑에' → '아래' 금지)
- 임의로 조사 추가 금지 (예: '수능 공부' → '수능 공부를' 금지)
- 어색하거나 비문이어도 의미가 통하면 수정하지 마
- 임의로 쉼표나 따옴표 삽입 금지

📌 **자주 나오는 오류 예시 (✅ 아래 예시대로 반드시 수정, 이미 올바른 경우는 그대로 둬)**
- '그치만' → '그렇지만'
- '스험생활' → '수험생활'
- '평행 이동' → '평행이동'
- '바뀌어 가지고' → '바뀌어가지고'
- '터득 해가지고' → '터득해가지고'
- '쪼끔' → '쪼금'
- '썸네일' → '섬네일'
- '문제풀이'와 '문제 풀이'는 둘 다 맞음 → 수정하지 마
- '맞는 듯 해' → '맞는듯해'
- '없는 듯' → '없는듯'
- '해 주셨다' → '해주셨다'
- '도와 주셔서' → '도와주셔서'
- '되주고자' → '돼주고자'
- '구성운' → '구성원'
- '어떡해설쩡' → '어떻게 설정'
- '학교급간' → '학교급 간'
- '지가' → '자기가' (단, ‘의지가’는 그대로 둠)
- '그리구간격' → '그리고 간격'
- '거이죠' → '것이죠'
- '고치구나서' → '고치고 나서'
- '시간 깨 줄어들어요' → '시간 꽤 줄어들어요.'
- '이해못할' → '미해 못 할'
- '거리면 괜찮다' → '걸리면 괜찮다.'
- '받았엇어' → '받았었어.'
- '감성있다고 한' → '감성 있다고 한'
- '잘안되요.' → '잘 안돼요'
- '받아 드릴' → '받아드릴'
- '불가능 하니가' → '불가능하니까'
- '공매도는다있잖아.' → '공매도는 다 있잖아.'
- '같이나가시는데' → '같이 나가시는데'
- '전문항' → '전 문항'
- '아니구성적' → '아니고 성적'
- '고전적이미에서' → '고전적 의미'
- '안되구' → '안되고'
- '멘날' → '맨날'
- '응원해주셔서' → '응원해 주셔서'

📌 **올바르게 입력된 예시 (✅ 이미 올바르니 수정하지 마)**
- '바랬다', '계속하셔요', '좋은건', '가득가득', '맞춰보는', '세부정보', '밑줄하고'

출력 형식: 교정된 문장만 한 줄로 출력 (설명 없이)

입력문장: {{text}}"""

# 테스트 데이터 로딩
df = pd.read_csv("data/test.csv")
sample = [
    {"id": row["id"], "input": row["err_sentence"]}
    for _, row in df.iterrows()
    if pd.notnull(row["err_sentence"])
]

# LLM 호출
async def call_with_limit(client, messages, input_text, max_retries=3):
    async with semaphores[client]:
        for attempt in range(1, max_retries + 1):
            try:
                response = await client.chat.completions.create(
                    model="solar-pro",
                    messages=messages,
                    temperature=0.0,
                    top_p=1.0
                )
                await asyncio.sleep(0.7)  # 요청 간 딜레이 추가
                return response.choices[0].message.content.strip()
            except Exception as e:
                print(f"⚠️ API 오류 (시도 {attempt}/{max_retries}): {e}")
                if attempt == max_retries:
                    print(f"❌ 실패: {input_text[:50]}...")
                    return "[ERROR]"
                await asyncio.sleep(1.5 * attempt)  # 재시도 대기 시간 증가

# 예측 및 저장
async def generate_submission(output_path="data/submission.csv", batch_size=100):
    results = []

    async def worker(row):
        client = next(client_cycle)
        messages = format_prompt(template_str, row["input"])
        pred = await call_with_limit(client, messages, row["input"])
        return {
            "id": row["id"],
            "err_sentence": row["input"],
            "cor_sentence": pred
        }

    for i in tqdm(range(0, len(sample), batch_size), desc="📤 테스트셋 예측 진행 중"):
        batch = sample[i:i+batch_size]
        tasks = [worker(row) for row in batch]
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)

    # 저장
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(output_path, index=False)
    print(f"\n✅ 예측 완료: {len(results)}개 문장 → '{output_path}' 저장됨")

# ▶ 실행
await generate_submission()

📤 테스트셋 예측 진행 중: 100%|██████████| 109/109 [12:23<00:00,  6.82s/it]


✅ 예측 완료: 10870개 문장 → 'data/submission.csv' 저장됨


In [ ]:
TEMPLATES = {
    "CHECK_SINGLE": {
        "step1": (
        "다음 문장에서 **오탈자 / 문장부호 / 띄어쓰기** 오류를 모두 교정해줘."

        "📌 **교정 원칙**"
        "1. **오탈자(철자 오류)**"
        "자판 입력 실수나 자음/모음 혼동을 고쳐줘 (예: '됬다' → '됐다')"
        "모양이나 발음이 비슷한 단어(예: 외/왜, 믈론/물론 등)를 고려하되,"
        "① 철자 오류인지 확인하고, ② 문장 의미에 비추어 자연스럽고 적절한 단어일 경우에만 수정해"
        "문맥상 부적절하거나 존재하지 않는 단어는 자연스러운 단어로 바꿔줘"

        "2. **띄어쓰기**"
        "의미 단위에 맞춰 문법적으로 띄어쓰기 교정"
        "아래 단어는 반드시 **띄어 써야 해**: 다음과 같이 사용하면 돼"
        "조사, 어미, 접사 사이를 임의로 띄우지 마."
        "단어를 붙여야 할 경우 정확한 어절 단위로 붙여줘."
        "오타나 문장부호는 수정하지 마."
        "문체나 표현 방식은 유지해줘."
        "두가지 단어가합쳐진 단어는 때에따라서 붙여쓸 수도있고 띄어쓸 수 도있어. 다음과같은 규칙에따라 사용해줘"
        "아래 단어는 반드시 **띄어 써야 해**: 다음과 같이 사용하면 돼"
        "'조회 수', '전공 수업', '행동 강령', '수능 문학', '샤프 심', '백지 복습', '후 순위', '교직 이수', '배면 발광'" 
        "'전면 발광', '지역 페이', '초등 교사', '가짜 뉴스', '자음군 단순화', '기숙 학원'. '만국 공법', '공부 의욕', '노인 성비', "
        "'극 초반', '연극 영화과'"

        "아래 단어는 반드시 **붙여 써야 해**: 다음과 같이 사용하면 돼."
        " '수시접수', '시험기간', '구조독해', '동해바다', '학교생활', '공부시간', '정정기간', '인구수', '의사분', '회전주기',"
        "'공부자극', '악성코드', '나비효과', '필기노트', '주격조사', '수학학원', '인지심리학', '대학과정','편가르기', '학습방향', '치고는요', "
        "'교수되신', '회전관성', '파생접사','과잠사진', '사형제도', ''통합검색 "
        "관용적 표현(‘해주셨다’, ‘도와주셔서’, ‘한밤중’ 등)은 붙여 써"
        "'듯’ 표현은 다음처럼 붙여 써야 해: ‘맞는듯해’, ‘없는듯’, ‘괜찮을듯하다’"
        "-해가지고’ 표현은 붙여 써야 해: ‘바뀌어가지고’, ‘터득해가지고’"
        "‘주다’가 보조동사로 쓰인 경우는 붙여 써: ‘도와주셨다’, ‘해주셔서’"

        "3. **문장부호**"
        "문장 끝에 종결 부호가 없을 경우에 마침표(.)을 추가해줘"
        "쉼표(,)는 원문에 있는 경우만 유지하고 새로 삽입하지 마"
        "정식 문장부호가 아닌 (...), (...?), (..?), (...!) 등은 교정하지 말고 유지해."
        "이외 문장부호 추가 또는 변경 금지"

        "**교정 순서**"
        "① 단어 내부 철자 오류(오탈자) → ② 띄어쓰기 → ③ 문장부호"

        "**금지 사항**"
        "표현 방식, 말투, 격식 수준 변경 금지 ('선생' → '선생님' 이렇게변경하는거 하지마., '거' → '것' 이렇게변경하는거 하지마.)"
        "같은 의미라도 조사나 어미, 단어 자체 변경 금지 (예: '있을까요' → '있으실까요' , '밑에' -> '아래' 이렇게 변경하지마)"
        "임의로 조사 추가하지 마 (조사: 이,가,을,를,에, 에서, 에게, 한테, 와,과, 하고, 외, 은, 는, 도, 만, 까지, 마저, 조차, 뿐) 추가 금지야. 문장에있는 조사 오타나 오류만  교정해."
        "문장이 다소 어색하거나 비문이어도 의미는 유지하고 오류만 수정해"
        "문장 내 임의로 쉼표(,)나 따옴표(\"') 삽입 금지"

        "📌 **자주 나오는 오류 예시 (입력 = 왼쪽, 교정 = 오른쪽)**, 오른쪽과같이 교정해줘."
"""
'그치만' = '그렇지만'
'스험생활' = '수험생활'
'평행 이동' = '평행이동'
'바뀌어 가지고' = '바뀌어가지고'
'터득 해가지고' = '터득해가지고'
'쪼끔' = '쪼금', '조금'은 그대로 유지
'썸네일' = '섬네일'
'문제풀이'와 '문제 풀이'는 모두 맞는 표현 (수정 금지)
'맞는 듯 해' = '맞는듯해'
'없는 듯' = '없는듯'
'해 주셨다' = '해주셨다'
'도와 주셔서' = '도와주셔서'
'되주고자' = '돼주고자'
'구성운' = '구성원'
'어떡해설쩡' = '어떻게 설정'
'학교급간' = '학교급 간'
'지가' = '자기가'  (단, '의지가' 등은 제외)
'그리구간격' = '그리고 간격'
'거이죠' = '것이죠'
'고치구나서' = '고치고 나서'
'시간 깨 줄어들어요' = '시간 꽤 줄어들어요.'
'이해못할' = '미해 못 할'
'거리면 괜찮다' = '걸리면 괜찮다.'
'받았엇어' = '받았었어.'
'감성있다고 한' = '감성 있다고 한'
'잘안되요.' = '잘 안돼요'
'받아 드릴' = '받아드릴'
'불가능 하니가' = '불가능하니까'
'같이나가시는데' = '같이 나가시는데'
'전문항' = '전 문항'
'아니구성적' = '아니고 성적'
'고전적이미에서' = '고전적 의미'
'안되구' = '안되고'
'멘날' = '맨날'
'응원해주셔서' = '응원해 주셔서'
'숨참구' = '숨 참고'
'한거음 한거음' = '한걸음 한걸음'
'이거이' = '이것이'
'트린' = '틀린'
'재심장' = '제 심장'
'카페인음료순' = '카페인 음료수는'
'가돼면' = '가 되면'
'어디가지' = '어디 가지'
'멀' = '뭘'
'점' = '좀'
'존재하는거라' = '존재하는 거라'
'먼가' = '뭔가'
"""
        "📌 **올바르게 입력되는 예시 (올바르게 입력되는 예시이므로 교정 하지마.)**"
        "'바랬다', '계속하셔요', '좋은건', '가득가득', '맞춰보는', ''세부정보, '밑줄하고', '거임', '몇인지', '잘하면', '맞다면', "

        "출력 형식: 교정된 문장만 한 줄로 출력 (설명 없이)"
        "입력문장: {text}"
        )
    }
}